## Lecture 5: Managing experiments and process data
### 0. Overview
- tf.train.Saver() class
- visualize summary statistics during training
- random seed (TF) and random state (NP)
- reading data in TF

### 1. tf.trainSaver()
- save the graph's variables (model parameters) in binary files
- enable restoration/retainment of model after a certain number of steps
- **checkpoint**: the step at which you save your graph's variables

    tf.train.Saver.save(sess, save_path, global_step=None, latest_filename=None, meta_graph_suffix='meta', write_meta_graph=True, write_state=True)

- example: save variables of the graph after every 1000 training steps

        # define model

        # create a saver object
        saver = tf.train.Saver()

        # launch a session to compute the graph
        with tf.Session as sess:
            # actual training loop
            for step in range(training_steps):
                sess.run([optimiser])

                if (step + 1) % 1000 ==0:
                    saver.save(sess, 'checkpoint_directory/model_name',
                              global_step=model.global_step)

- helpful to append the # of training steps the model has gone through in a variable `global_step` (initialize to be 0, NOT trainable)

        self.global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
        
        # pass global_step as a parameter to the optimizer
        # so it is icremented by one with each training step
        self.optimizer = tf.train.GradientDescentOptimizer(self.lr).minimize(self.loss, global_step=self.global_step)
        


- training loop for word2vec:

        self.global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

        self.optimizer = tf.train.GradientDescentOptimizer(self.lr).minimize(self.loss, global_step=self.global_step)

        saver = tf.train.Saver() # defaults to saving all variables

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            average_loss = 0.0
            writer = tf.summary.FileWriter('./improverd_graph', sess.graph)
            for index in xrange(num_train_steps):
                batch = batch_gen.next()
                loss_batch, _ = sess.run([model.loss, model.optimizer],
                                        feed_dict={model.center_words: batch[0],
                                                   model.target_words: batch[1]})
                average_loss += loss_batch
                if (index + 1) % 1000 ==0:
                    saver.save(sess, 'checkpoints/skip-gram', global_step=model.global_step)



- restore **variables** (not the entire graph, need to create graph ourselves):
    
        # check if there is a checkpoint
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
        
        if ckpt and ckpt.model_checkpoint_path:
            # restore variables
            saver.restore(sess, 'checkpoints/skip-gram-10000')
            
    - stores all variables by default (recommended)
    - store selected variables:

            v1 = tf.Variable(..., name='v1')
            v2 = tf.Variable(..., name='v2')

            # pass the variables as a dict
            saver = tf.train.Saver({'v1': v1, 'v2': v2})

            # pass the variables as a list
            saver = tf.train.Saver([v1, v2])

            # passing a list is squivalent to passing a dict with the variable op names # as keys
            saver = tf.train.Saver({v.op.name: v for v in [v1, v2]})
            

### 2.  visualize summary statistics during training
#### 2.1 some popular statistics to visualize:
- loss
- average loss
- accuracy

#### 2.2 walkthrough
- namescope to hold summary ops

        def _create_summaries(self):
            with tf.name_scope("summaries"):
                tf.summary.scalar("loss", self.loss)
                tf.summary.scalar("accuracy", self.accuracy)
                tf.summary.histogram("histogram loss", self.loss)

                # because you have several summaries, we should merge them all
                # into one op to make it easier to manage
                self.summary_op = tf.summary.merge_all()

- because it's an op, have to execute it:

        loss_batch, _, summary = sess.run([model.loss, model.optimizer, model.summary_op],
                                          feed_dict=feed_dict)
                                          
- write the summary to file (using the same FileWriter object we created to visual our graph)

        writer.add_summary(summary, global_step=step)
        
- run TensorBoard and go to http://localhost:6006/, check Scalars page
- can compare the progress made with different optimizers or different parameters
- visualize the statistics as images

        tf.summary.image(name, tensor, max_outputs=3, collections=None)
        

### 3. Control randomization
TensorFlow doesn’t allow to you to get random state the way numpy does.

#### 3.1 2 ways to get stable results in randomization:
- Set random seed at operation level
        
        my_var = tf.Variable(tf.truncated_normal((-1.0,1.0), stddev=0.1, seed=0))
        
   **session keeps track of random state**, each new session will start the random state all over again
   
        c = tf . random_uniform ([], - 10 , 10 , seed = 2)
        with tf . Session () as sess:
        print sess . run ( c) # >> 3.57493
        print sess . run ( c) # >> -5.97319


        c = tf . random_uniform ([], - 10 , 10 , seed = 2)
        with tf . Session () as sess:
        print sess . run ( c) # >> 3.57493
        with tf . Session () as sess:
        print sess . run ( c) # >> 3.57493

    with operation-level random seed, each op keeps its own seed
    
        c = tf.random_uniform([], -10, 10, seed=2)
        d = tf.random_uniform([], -10, 10, seed=2)
        
        with tf.Session as sess:
            print(sess.run(c)) # >> 3.57493
            print(sess.run(d)) # >> 3.57493


- Set random seed at graph level with tf.Graph.seed  
If you don’t care about the randomization for each op inside the graph, but just want to be able
to replicate result on another graph (so that other people can replicate your results on their own
graph), you can use tf.set_random_seed instead.  
For example, you have two models a.py and b.py that have identical code:


In [ ]:
import tensorflow as tf
tf.set_random_seed(2)
c = tf.random_uniform([], -10 , 10)
d = tf.random_uniform([], -10 , 10)
with tf.Session() as sess:
print sess. run (c)
print sess. run (d)

Without graph level seed, running python a.py and b.py will return 2 completely different results,
but with tf.set_random_seed, you will get two identical results:

    $ python a . py
    >> - 4.00752
    >> - 2.98339
    $ python b . py
    >> - 4.00752
    >> - 2.98339

### 4. Reading data in TensorFlow
#### 4.1 through feed_dict
Feed_dict will first send data from the storage system to the client, and then
from client to the worker process. This will cause the data to slow down, especially if the client is
on a different machine from the worker process.

#### 4.2 through readers
Allow us to load data directly into the worker process
- `tf.TextLineReader`: Outputs the lines of a file delimited by newlines.  
e.g. text files, CSV files
- `tf.FixedLengthRecordReader`: Outputs the entire file when all files have same fixed lengths  
e.g. each MNIST file has 28 x 28 pixels , CIFAR-10 32 x 32 x 3
- `tf.WholeFileReader`: Outputs the entire file content
- `tf.TFRecordReader`: Reads samples from TensorFlow's own binary format (TFRecord)
- `tf.ReaderBase`: Allows you to create your own readers  
more in [Lecture 9](09_Input Pipeline.ipynb)

#### 4.3 load data using constants
only use this if want graph to be seriously bloated and un-runnable